<a href="https://colab.research.google.com/github/lani009/IDS-DataMining/blob/main/%5BDM%5DApriori.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time

In [2]:
data = pd.read_csv('DM_data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25192 entries, 0 to 25191
Data columns (total 40 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   duration                     25192 non-null  int64  
 1   protocol_type                25192 non-null  int64  
 2   service                      25192 non-null  int64  
 3   flag                         25192 non-null  int64  
 4   src_bytes                    25192 non-null  int64  
 5   dst_bytes                    25192 non-null  int64  
 6   land                         25192 non-null  int64  
 7   wrong_fragment               25192 non-null  int64  
 8   hot                          25192 non-null  int64  
 9   num_failed_logins            25192 non-null  int64  
 10  logged_in                    25192 non-null  int64  
 11  num_compromised              25192 non-null  int64  
 12  root_shell                   25192 non-null  int64  
 13  su_attempted    

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [4]:
from mlxtend.frequent_patterns import apriori,association_rules

In [5]:
sc = StandardScaler() 
sc_data = sc.fit_transform(data)

sc_df = pd.DataFrame(sc_data, columns=data.columns)
sc_df.head(n=10)

#StandardScaler로 data scaling

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class,index_num
0,-0.113551,-0.444009,-1.399448,0.744553,-0.009889,-0.039310,-0.00891,-0.091223,-0.091933,-0.02622,-0.807626,-0.021873,-0.039377,-0.027665,-0.021724,-0.027808,-0.018905,-0.043917,-0.09599,-0.720244,-0.354628,-0.640142,-0.633978,-0.372186,-0.373098,0.772109,-0.349282,-0.373886,-0.328634,-0.813985,-0.779157,-0.280673,0.073120,-0.287993,-0.641804,-0.627365,-0.221668,-0.374281,-0.934425,-1.731982
1,-0.113551,1.325565,0.780883,0.744553,-0.010032,-0.039310,-0.00891,-0.091223,-0.091933,-0.02622,-0.807626,-0.021873,-0.039377,-0.027665,-0.021724,-0.027808,-0.018905,-0.043917,-0.09599,-0.624317,-0.368427,-0.640142,-0.633978,-0.372186,-0.373098,-1.320567,0.490836,-0.373886,0.732059,-1.030895,-1.157831,2.764403,2.375620,-0.287993,-0.641804,-0.627365,-0.385140,-0.374281,-0.934425,-1.731845
2,-0.113551,-0.444009,-1.377199,-0.917300,-0.010093,-0.039310,-0.00891,-0.091223,-0.091933,-0.02622,-0.807626,-0.021873,-0.039377,-0.027665,-0.021724,-0.027808,-0.018905,-0.043917,-0.09599,0.334947,-0.299430,1.595477,1.600209,-0.372186,-0.373098,-1.388806,0.042773,-0.373886,0.732059,-0.804947,-0.935081,-0.173828,-0.478183,-0.287993,1.603834,1.614454,-0.385140,-0.374281,1.070177,-1.731707
3,-0.113551,-0.444009,0.780883,0.744553,-0.009996,0.052473,-0.00891,-0.091223,-0.091933,-0.02622,1.238197,-0.021873,-0.039377,-0.027665,-0.021724,-0.027808,-0.018905,-0.043917,-0.09599,-0.694082,-0.313230,-0.193018,-0.187141,-0.372186,-0.373098,0.772109,-0.349282,-0.373886,-1.540854,1.264742,1.069663,-0.440940,-0.380894,0.073759,-0.574435,-0.604947,-0.385140,-0.342768,-0.934425,-1.731570
4,-0.113551,-0.444009,0.780883,0.744553,-0.010010,-0.034582,-0.00891,-0.091223,-0.091933,-0.02622,1.238197,-0.021873,-0.039377,-0.027665,-0.021724,-0.027808,-0.018905,-0.043917,-0.09599,-0.476067,0.059355,-0.640142,-0.633978,-0.372186,-0.373098,0.772109,-0.349282,-0.023115,0.732059,1.264742,1.069663,-0.440940,-0.478183,-0.287993,-0.641804,-0.627365,-0.385140,-0.374281,-0.934425,-1.731432
5,-0.113551,-0.444009,-1.377199,-2.025203,-0.010093,-0.039310,-0.00891,-0.091223,-0.091933,-0.02622,-0.807626,-0.021873,-0.039377,-0.027665,-0.021724,-0.027808,-0.018905,-0.043917,-0.09599,0.317506,-0.120038,-0.640142,-0.633978,2.765176,2.729322,-1.138595,-0.013235,-0.373886,0.732059,-0.868212,-1.001906,-0.066984,-0.478183,-0.287993,-0.641804,-0.627365,2.884296,2.777041,1.070177,-1.731295
6,-0.113551,-0.444009,-1.377199,-0.917300,-0.010093,-0.039310,-0.00891,-0.091223,-0.091933,-0.02622,-0.807626,-0.021873,-0.039377,-0.027665,-0.021724,-0.027808,-0.018905,-0.043917,-0.09599,0.709933,-0.258032,1.595477,1.600209,-0.372186,-0.373098,-1.388806,-0.013235,-0.373886,0.732059,-0.958592,-1.068731,-0.173828,-0.478183,-0.287993,1.603834,1.614454,-0.385140,-0.374281,1.070177,-1.731157
7,-0.113551,-0.444009,-1.377199,-0.917300,-0.010093,-0.039310,-0.00891,-0.091223,-0.091933,-0.02622,-0.807626,-0.021873,-0.039377,-0.027665,-0.021724,-0.027808,-0.018905,-0.043917,-0.09599,0.282624,-0.161436,1.595477,1.600209,-0.372186,-0.373098,-1.184088,-0.013235,-0.373886,0.732059,-0.904364,-1.024181,-0.066984,-0.478183,-0.287993,1.603834,1.614454,-0.385140,-0.374281,1.070177,-1.731019
8,-0.113551,-0.444009,0.780883,-0.917300,-0.010093,-0.039310,-0.00891,-0.091223,-0.091933,-0.02622,-0.807626,-0.021873,-0.039377,-0.027665,-0.021724,-0.027808,-0.018905,-0.043917,-0.09599,1.616874,-0.064840,1.595477,1.600209,-0.372186,-0.373098,-1.297820,-0.069243,-0.373886,0.732

In [6]:
def encode_units(x):
  if x <= 0 :
    return 0
  if x > 0 :
    return 1

train_df = sc_df.applymap(encode_units)

train_df.head(n=10)

#classification을 위해 scaling 시킨 data들을 음수면 0, 양수면 1로 encoding

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class,index_num
0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,1,0,1,0,0,0,0,0,1,1,0,0,1,0
3,0,0,1,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,1,0,0,0,0,0,0
4,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,1,1,1,0
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,1,0
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,1,0
8,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,1,0
9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,1,0


In [7]:
data_X = train_df.drop(columns = ["index_num"])

X_train, X_test = train_test_split(data_X, test_size=0.3, random_state=42)
print(X_train.shape, X_test.shape)

#train data와 test data를 7:3 의 비율로 split

(17634, 39) (7558, 39)


In [8]:
df = pd.DataFrame(X_train, columns=data.drop(columns = ["index_num"]).columns)

df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
741,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,1,0,1,0,0,0,0,0,1,1,0,0,1
411,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,1,1,1
17841,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,1,0,1,0,0,0,0,0,0,0,1,1,1
20962,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,1,1,0,1,1,0,0,0,0,1
17790,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,1


In [9]:
frequent_itemsets = apriori( df, min_support = 0.27, use_colnames=True)
result_desc = frequent_itemsets.sort_values(['support'],ascending =[False])
result_desc

,support,itemsets
7,0.642225,(dst_host_count)
6,0.622547,(same_srv_rate)
0,0.618634,(service)
1,0.611773,(flag)
20,0.567143,"(flag, same_srv_rate)"
...,...,...
75,0.273789,"(class, dst_host_srv_serror_rate, dst_host_ser..."
94,0.273733,"(class, dst_host_srv_serror_rate, serror_rate,..."
95,0.273676,"(class, dst_host_srv_serror_rate, srv_serror_r..."
102,0.273676,"(class, srv_serror_rate, serror_rate, dst_host..."


In [10]:
rules = association_rules(result_desc , metric = "confidence" , min_threshold = 0.9)
rules = rules.sort_values(['confidence','lift'], ascending=[False , False])
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
220,"(srv_serror_rate, dst_host_serror_rate)",(serror_rate),0.275264,0.286152,0.275264,1.000000,3.494649,0.196497,inf
250,"(class, srv_serror_rate, dst_host_serror_rate)",(serror_rate),0.274413,0.286152,0.274413,1.000000,3.494649,0.195889,inf
287,"(dst_host_srv_serror_rate, srv_serror_rate, ds...",(serror_rate),0.273959,0.286152,0.273959,1.000000,3.494649,0.195565,inf
339,"(class, dst_host_srv_serror_rate, srv_serror_r...",(serror_rate),0.273676,0.286152,0.273676,1.000000,3.494649,0.195363,inf
215,"(dst_host_srv_serror_rate, serror_rate)",(srv_serror_rate),0.275377,0.283600,0.275320,0.999794,3.525369,0.197223,3478.839061
...,...,...,...,...,...,...,...,...,...
86,"(service, same_srv_rate, logged_in)","(dst_host_same_srv_rate, flag)",0.353351,0.470795,0.318759,0.902102,1.916125,0.152403,5.405698
197,"(dst_host_same_srv_rate, flag, logged_in)","(service, dst_host_srv_count, same_srv_rate)",0.331802,0.344732,0.299195,0.901726,2.615733,0.184812,6.667782
75,"(service, flag, logged_in)",(dst_host_same_srv_rate),0.355450,0.498809,0.320347,0.901244,1.806792,0.143046,5.075064
152,"(dst_host_same_srv_rate, logged_in)","(service, dst_host_srv_count, same_srv_rate)",0.334524,0.344732,0.301463,0.901170,2.614118,0.186142,6.630236


In [11]:
rules_list = rules[rules['consequents'] == {"class"}]
rules_list

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
314,"(dst_host_srv_serror_rate, serror_rate, dst_ho...",(class),0.274016,0.466542,0.273733,0.998965,2.141212,0.145893,515.533900
327,"(dst_host_srv_serror_rate, srv_serror_rate, ds...",(class),0.273959,0.466542,0.273676,0.998965,2.141212,0.145862,515.427209
341,"(serror_rate, dst_host_srv_serror_rate, srv_se...",(class),0.273959,0.466542,0.273676,0.998965,2.141212,0.145862,515.427209
246,"(srv_serror_rate, dst_host_serror_rate)",(class),0.275264,0.466542,0.274413,0.996910,2.136806,0.145991,172.627039
251,"(serror_rate, srv_serror_rate, dst_host_serror...",(class),0.275264,0.466542,0.274413,0.996910,2.136806,0.145991,172.627039
236,"(dst_host_srv_serror_rate, srv_serror_rate)",(class),0.275604,0.466542,0.274583,0.996296,2.135492,0.146002,144.033685
264,"(dst_host_srv_serror_rate, serror_rate)",(class),0.275377,0.466542,0.274356,0.996293,2.135485,0.145881,143.915139
269,"(serror_rate, dst_host_srv_serror_rate, srv_se...",(class),0.275320,0.466542,0.274300,0.996292,2.135483,0.145851,143.885502
241,"(serror_rate, dst_host_serror_rate)",(class),0.276341,0.466542,0.274526,0.993433,2.129355,0.145602,81.235665
308,"(dst_host_srv_serror_rate, dst_host_serror_rate)",(class),0.276625,0.466542,0.273789,0.989750,2.121460,0.144732,52.044171


In [12]:
col = rules_list['antecedents']
col

314    (dst_host_srv_serror_rate, serror_rate, dst_ho...
327    (dst_host_srv_serror_rate, srv_serror_rate, ds...
341    (serror_rate, dst_host_srv_serror_rate, srv_se...
246              (srv_serror_rate, dst_host_serror_rate)
251    (serror_rate, srv_serror_rate, dst_host_serror...
236          (dst_host_srv_serror_rate, srv_serror_rate)
264              (dst_host_srv_serror_rate, serror_rate)
269    (serror_rate, dst_host_srv_serror_rate, srv_se...
241                  (serror_rate, dst_host_serror_rate)
308     (dst_host_srv_serror_rate, dst_host_serror_rate)
233                           (dst_host_srv_serror_rate)
228                       (serror_rate, srv_serror_rate)
210                                    (srv_serror_rate)
207                                        (serror_rate)
232                               (dst_host_serror_rate)
Name: antecedents, dtype: object

In [13]:
col.to_csv('./col_list.csv')

In [14]:
test = pd.DataFrame(X_test, columns=data.drop(columns = ["index_num"]).columns)
test.head(n=10)

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
19064,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,1,0,1,0,0,0,0,0,1
11127,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,1,1,0,0,1,0,0,1,1,0
6517,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,1
2973,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0
13339,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,1,0,0,1,0,0,0,1,1,0,0,1
19289,0,0,1,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,1,0,0,1,0,0,0,0,0
2166,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,1,0,0,0,0,0,0,0,0
5548,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
10887,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,1,0,0,0,0,0,0,0,0
2222,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,1


In [15]:
col = ['dst_host_srv_serror_rate', 'srv_serror_rate', 'serror_rate', 'dst_host_serror_rate']

In [17]:
idx_b = test[(test['dst_host_srv_serror_rate'] == 0) | (test['srv_serror_rate'] == 0) | (test['serror_rate'] == 0) | (test['dst_host_serror_rate'] == 0)].index
test_df = test.drop(idx_b)

idx_class = test_df[test_df['class'] == 0 ].index
test_err = test_df.drop(idx_class)

print(test_df.shape)
print(test_err.shape)

(2121, 39)
(2120, 39)


In [27]:
idx_a = test[(test['dst_host_srv_serror_rate'] == 1) & (test['srv_serror_rate'] == 1) & (test['serror_rate'] == 1) & (test['dst_host_serror_rate'] == 1)].index
test_df = test.drop(idx_a)

idx_class = test_df[test_df['class'] == 1 ].index
test_err = test_df.drop(idx_class)

print(test_df.shape)
print(test_err.shape)

(5437, 39)
(4041, 39)




               |   Prediction of Attack     |  Prediction of Non-Attack
---
    Attack     |    True Positive : 2020    |    False Negative : 1396
---
    Non-Attack |    False Positive : 1      |    True Negative : 4041




**Apriori Test**



*   Accuracy = 80.19%
*   Precision = 99.95%
*   Recall = 59.13%
*   Fallout = 0.02%
*   F-score = 74.3